In [1]:
import numpy as np
import random as random
import math

    
def multinomial(lst):
    res, i = 1, sum(lst)
    i0 = lst.index(max(lst))
    for a in lst[:i0] + lst[i0+1:]:
        for j in range(1,a+1):
            res *= i
            res //= j
            i -= 1
    return res

def c_norm(a_b,alpha,entry):
    m = (alpha+1)*(alpha+2)//2
    A = np.zeros((m+1,m+1))
    for i in range(m+1):
        count = 0
        for j in range(alpha+1):
            for k in range(alpha+1-j):
                A[i][count] = (multinomial([alpha-j-k,j,k])*a_b[i]**(alpha-j-k)*a_b[m+1+i]**j/((a_b[i]**2+a_b[m+1+i]**2 + 1)**(alpha/2)))
                count = count + 1
                #print((i,j,k,count))
    b = np.array([0 for i in range(m+1)])
    b[entry]=1

    epsilon = 0.000000001
    
    x = np.linalg.solve(A + epsilon * np.identity(m+1), b)
    
    return sum(abs(i) for i in x)


def optimizeWithInit(a_b,alpha,entry):
    res = op.minimize(c_norm, a_b, method='BFGS', tol=1e-6, options={'maxiter': 100}, args=(alpha, entry))
    aopt = res.x
    copt = res.fun
    return copt


In [ ]:
import scipy.optimize as op
from joblib import Parallel, delayed
import time

for alpha in range(3, 4):
    start_time = time.time()
    print('For the ' + str(alpha) + '-tensor:')
    entry = 6
    
    best_aopt = []
    best_copt = 9999999999

    for jj in range(1,500):
    
        sample = 15*[[np.random.normal() for i in range(22)] for i in range(40)]

        allcopt = Parallel(n_jobs=4)(delayed(optimizeWithInit)(a_b,alpha,entry) for a_b in sample)
        
        if min(allcopt) < best_copt:
              best_copt = min(allcopt)
              
        print(best_copt)

    print('Optimal a vector:', np.round(best_aopt, 3))
    print('Optimal Coefficient:', np.round(best_copt, 3))
    print("--- %s seconds ---" % (time.time() - start_time))
    print('\n')